In [ ]:
import numpy as np
import glob
import shutil
import pandas as pd
import cv2
import os

In [ ]:
# YOLOv5
!git clone https://github.com/ultralytics/yolov5
!mv yolov5/* ./
!pip install -r requirements.txt
!pip uninstall -y wandb

In [ ]:
# apex
# !git clone https://github.com/NVIDIA/apex
# !pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
classes = ["A10","B1","B2","B52", "Be200", "C130","C17","C5", "E2","EF2000",
           "F117","F14","F15","F16","F18","F22","F35","F4",
           "JAS39","MQ9","Mig31","Mirage2000","RQ4","Rafale",
           "SR71","Su57","Tu160","Tu95","U2","US2", "V22","XB70","YF23","J20"]
classes = np.array(classes)

In [ ]:
csv_paths = glob.glob('../input/militaryaircraftdetectiondataset/dataset/*.csv')
jpg_paths = glob.glob('../input/militaryaircraftdetectiondataset/dataset/*.jpg')
csv_paths.sort()
jpg_paths.sort()
print(csv_paths[0])
print(jpg_paths[0])
print(len(csv_paths), len(jpg_paths))

In [ ]:
os.makedirs('yolov5/data/train/images', exist_ok=True)
os.makedirs('yolov5/data/train/labels', exist_ok=True)

os.makedirs('yolov5/data/valid/images', exist_ok=True)
os.makedirs('yolov5/data/valid/labels', exist_ok=True)

os.makedirs('yolov5/data/test/images', exist_ok=True)
os.makedirs('yolov5/data/test/labels', exist_ok=True)

In [ ]:
n_images = len(csv_paths)
n_split = n_images//20

for i, (csv_path, jpg_path) in enumerate(zip(csv_paths, jpg_paths)):
    annotations = np.array(pd.read_csv(csv_path))
    
    if i < n_split:
        shutil.copy(jpg_path, 'yolov5/data/test/images/' + os.path.basename(jpg_path))
        txt_file_path = 'yolov5/data/test/labels/' + os.path.basename(csv_path)[:-4]+'.txt'
    elif n_split <= i < n_split*5:
        shutil.copy(jpg_path, 'yolov5/data/valid/images/' + os.path.basename(jpg_path))
        txt_file_path = 'yolov5/data/valid/labels/' + os.path.basename(csv_path)[:-4]+'.txt'
    else:
        shutil.copy(jpg_path, 'yolov5/data/train/images/' + os.path.basename(jpg_path))
        txt_file_path = 'yolov5/data/train/labels/' + os.path.basename(csv_path)[:-4]+'.txt'
    
    with open(txt_file_path, mode='w') as f:

        for annotation in annotations:
            width = annotation[1]
            height = annotation[2]
            class_name = annotation[3]
            xmin = annotation[4]
            ymin = annotation[5]
            xmax = annotation[6]
            ymax = annotation[7]
            x_center = 0.5*(xmin+xmax)
            y_center = 0.5*(ymin+ymax)
            b_width = xmax - xmin
            b_height= ymax - ymin
            class_num = np.where(classes==class_name)[0][0]
            output_string = '{} {} {} {} {}\n'.format(class_num,
                                                    x_center/width,
                                                    y_center/height,
                                                    b_width/width,
                                                    b_height/height)
            f.write(output_string)
print(len(glob.glob('yolov5/data/test/labels/*.txt')))
print(len(glob.glob('yolov5/data/valid/labels/*.txt')))
print(len(glob.glob('yolov5/data/train/labels/*.txt')))

In [ ]:
!ls

In [ ]:
# train
'''
!python train.py --batch 2 \
                 --img 1024 \
                 --epochs 1 \
                 --data ../input/yolov5config/data.yaml \
                 --cfg ../input/yolov5config/yolov5x.yaml \
                 --hyp ../input/yolov5config/hyp.yaml \
                 --name military_aircraft_detection \
                 --weights ../input/yolov5config/yolov5x.pt
'''

In [ ]:
# detect by pretrained weights
'''
!python detect.py --weights ../input/yolov5config/34-classes-trained.pt \
                  --img 1024 \
                  --conf 0.2 \
                  --iou-thres 0.4 \
                  --agnostic-nms \
                  --augment \
                  --source ./yolov5/data/valid/images \
                  --name inference_valid
'''

!python detect.py --weights ../input/yolov5config/34-classes-trained.pt \
                  --img 1024 \
                  --conf 0.2 \
                  --iou-thres 0.4 \
                  --agnostic-nms \
                  --augment \
                  --source ./yolov5/data/test/images \
                  --name inference_test

In [ ]:
!mv ./runs ./output

In [ ]:
!rm -r ./yolov5
!rm -r ./utils
!rm -r ./data
!rm -r ./models
!rm -r ./weights
!rm ./*